In [3]:
%cd F:\project4


F:\project4


In [16]:
from application import db
from application.users.models import User
from cassandra.cqlengine.management import sync_table


In [17]:
db.get_session()
sync_table(User)

In [18]:
#User.create_user("honeysaxena525@gmail.com", password='abc123')

In [12]:
user_a = User.create_user("abc@gmail.com", password='abc123')


Exception: user already has account with this email

In [19]:
user_a.password

'$argon2id$v=19$m=65536,t=3,p=4$pgu5iS/0Ya2a8zxeCcNzBA$CncfEST4KOo0kv6i4kelt5lqM3mdCxjsqiotswJJdwI'

In [21]:
user_a.verify_password('abc123')

True

In [40]:
from pydantic import BaseModel, EmailStr, SecretStr, validator

In [36]:
class UserLoginSchema(BaseModel):
    email: EmailStr
    password: SecretStr    

In [38]:
post_data = {
    "email": "abc@abc.jin",
    "password": "abc123"
}

In [39]:
UserLoginSchema(**post_data)

UserLoginSchema(email='abc@abc.jin', password=SecretStr('**********'))

In [48]:
class UserSignupSchema(BaseModel):
    email: EmailStr
    password: SecretStr
    confirm_password: SecretStr
        
    @validator("email")
    def email_available(cls, v, values, **kwargs):
        q = User.objects.filter(email=v)
        if q.count() != 0:
            raise ValueError("Email is not Available")
        return v    
        
    @validator("confirm_password")
    def pass_match(cls, v, values, **kwargs):
        password = values.get("password")
        confirm_password = v
        if password != confirm_password:
            raise ValueError("password do not match")
        return v    

In [49]:
data = UserSignupSchema(email="abc@abc.com", password="abc123", confirm_password="abc123")

In [50]:
data.json()

'{"email": "abc@abc.com", "password": "**********", "confirm_password": "**********"}'

In [46]:
data.dict()

{'email': 'abc@abc.com',
 'password': SecretStr('**********'),
 'confirm_password': SecretStr('**********')}

In [47]:
data.dict()['password'].get_secret_value()

'abc123'